# Concurrency in `rs` vs `py` for the Transformers Vectorization Microservice

Make sure to spin up the docker compose stack where the services are running with `docker compose up --build`

## Define the methods of the run

In [1]:
import concurrent.futures
import json
import requests
import time
import typing

def send(obj: typing.Dict[str, typing.Any], s: requests.Session, port: int) -> float:
    start = time.time()
    res = s.post(f"http://localhost:{port}/vectors", json={
        "text": obj["raw"],
    })
    taken = time.time() - start
    print(f"Retrieved vector in {taken:.2f} seconds")
    if res.status_code != 200:
        print(f"Failed to retrieve vector for {obj['id']} with text {obj['raw']}")
        print(res.text)
    return taken


def load(objs: typing.List[str], port: int) -> None:
    session = requests.Session()
    for obj in objs:
        send(json.loads(obj), session, port)

def run(port: int) -> None:
    futures: typing.List[concurrent.futures.Future] = []
    objs: typing.List[str] = []
    with open("data/sphere.1M.jsonl", "r") as file:
        for i, obj in enumerate(file):
            objs.append(obj)
            if (i + 1) % 10000 == 0:
                break
    print(f"Loaded {len(objs)} objects")
    print("Starting benchmark")
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        for i in range(4):
            futures.append(executor.submit(load, objs, port))
        print("Scheduled all tasks, waiting for completion")
        concurrent.futures.wait(futures)

## Run against the rust service

In [2]:
run(3000)

Loaded 10000 objects
Starting benchmark
Scheduled all tasks, waiting for completion
Retrieved vector in 0.22 seconds
Retrieved vector in 0.22 seconds
Retrieved vector in 0.22 seconds
Retrieved vector in 0.22 seconds
Retrieved vector in 0.15 seconds
Retrieved vector in 0.15 seconds
Retrieved vector in 0.17 seconds
Retrieved vector in 0.32 seconds
Retrieved vector in 0.17 seconds
Retrieved vector in 0.32 seconds
Retrieved vector in 0.14 seconds
Retrieved vector in 0.19 seconds
Retrieved vector in 0.14 seconds
Retrieved vector in 0.11 seconds
Retrieved vector in 0.13 seconds
Retrieved vector in 0.23 seconds
Retrieved vector in 0.10 seconds
Retrieved vector in 0.06 seconds
Retrieved vector in 0.06 seconds
Retrieved vector in 0.61 seconds
Retrieved vector in 0.16 seconds
Retrieved vector in 0.12 seconds
Retrieved vector in 0.16 seconds
Retrieved vector in 0.13 seconds
Retrieved vector in 0.86 seconds
Retrieved vector in 0.13 seconds
Retrieved vector in 0.16 seconds
Retrieved vector in 0.17 

## Run against the python service

In [2]:
run(8080)

Loaded 10000 objects
Starting benchmark
Scheduled all tasks, waiting for completion
